In [1]:
from huggingface_hub import login

login()  # This will prompt you for your Hugging Face API token

**Importing Required Libraries**

In [46]:
import wikipedia
from sentence_transformers import SentenceTransformer
from transformers import LlamaTokenizer, LlamaForCausalLM

**Initialize Sentence Transformer for Embeddings**

In [47]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

**Fetch Wikipedia Article**

In [48]:
def fetch_wikipedia_article(query, num_sentences=5):
    try:
        summary = wikipedia.summary(query, sentences=num_sentences)
        return summary
    except wikipedia.exceptions.DisambiguationError as e:
        return wikipedia.summary(e.options[0], sentences=num_sentences)
    except wikipedia.exceptions.HTTPTimeoutError:
        return "Sorry, I couldn't retrieve information at the moment."
    except wikipedia.exceptions.RedirectError:
        return "Sorry, I encountered an issue retrieving the article."

**Initialize Llama Model and Tokenizer**

In [49]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

**Generate Response Using the Llama Model**

In [53]:
def generate_llama_response(article_content):
    tokenizer.pad_token = tokenizer.eos_token  # Set pad_token to eos_token

    # Truncate input to a maximum length for better compatibility
    max_input_length = 256  # Adjust as needed
    inputs = tokenizer(article_content, return_tensors="pt", padding=True, truncation=True, max_length=max_input_length)

    # Generate response with max_new_tokens instead of max_length
    output = model.generate(**inputs, max_new_tokens=100)  # Generate up to 100 new tokens
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

**Handle User Interaction**

In [51]:
def chatbot():
    print("Welcome to the Medical Chatbot! (Type 'exit' to quit)")

    while True:
        user_input = input("You: ")

        if user_input.lower() == "exit":
            print("Goodbye!")
            break

        print("Fetching relevant information...")

        # Fetch the relevant Wikipedia article based on user input
        article_content = fetch_wikipedia_article(user_input)

        # Generate response from the Llama model
        response = generate_llama_response(article_content)

        print(f"Bot: {response}")

In [54]:
if __name__ == "__main__":
    chatbot()

Welcome to the Medical Chatbot! (Type 'exit' to quit)
You: What are the symptoms of diabetes?
Fetching relevant information...
Bot: Type 3 diabetes is a proposed pathological linkage between Alzheimer's disease and certain features of type 1 and type 2 diabetes.  Specifically, the term refers to a set of common biochemical and metabolic features seen in the brain in Alzheimer's disease, and in other tissues in diabetes; it may thus be considered a "brain-specific type of diabetes."  It was recognized at least as early as 2005 that some features of brain function in Alzheimer's disease mimic those that underlie diabetes.  However, the concept of type 3 diabetes is controversial, and as of 2021 it was not a widely or generally recognized diagnosis.
Metabolic risk factors such as hyperglycaemia,  oxidative stress and lipid peroxidation are common processes thought to be contributors to the development of Alzheimer's disease in people with diabetes.  But while insulin resistance is a risk 